### Define jobname

In [ ]:
pdbid = '1kf1'
drug  = None
jobname = f'{pdbid}_{drug}' if not drug is None else pdbid
indexfile=f'{jobname}.simulate.ndx'

### Center protein for a solvated trajectory and put residues in the unit cell 
- Input and output are both solvated trajectories
- 1. make residues in the unit cell
- The sequence of commands here are used to generate the "process.solvated.traj" shell scripts for production runs

In [ ]:
def rmpbc(filename,indexfile='index.ndx',center=0,output=0, postfix='c'):
    file = ".".join(filename.split('.')[:-1]).split('_')[0]
    extend = filename.split('.')[-1]
    cmd1 = f'echo "{center} {output}" | gmx trjconv -s {file}.tpr -f {filename} -o {file}_{postfix}.{extend} -pbc res -center -boxcenter zero -n {indexfile} -ndec 5 -ur compact'
    print("##========================command=line========================##")
    print(cmd1,sep="\n")
    print("##======================end=command=line======================##")
    !{cmd1}

### Output non-water parts with best fit 
- The index file needs to be setup first
- The sequence of commands here are used to generate the "rms.traj" shell scripts for production runs

In [ ]:
def grep_and_fit(filename,indexfile='index.ndx',center=0,output=0, postfix='rms'):
    file = ".".join(filename.split('.')[:-1]).split('_')[0]
    extend = filename.split('.')[-1]
    cmd1 = f'echo "{center} {output}" | gmx trjconv -s rms.tpr -f {filename} -o {file}_{postfix}.{extend} -n {indexfile} -fit rot+trans -ndec 5'
    print("##========================command=line========================##")
    print(cmd1,sep="\n")
    print("##======================end=command=line======================##") 
    !{cmd1}

In [ ]:
def rmpdbc_and_grep_and_fit(i):
    rmpbc(f"md{i}.xtc", indexfile, center='DNA')
    grep_and_fit(f"md{i}_c.xtc", indexfile, center='DNA', output='Model')
    grep_and_fit(f"md{i}_c.xtc", indexfile, center='DNA', output='non-Water', postfix='noh')
import concurrent.futures

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for i in range(10):
        futures.append(executor.submit(rmpdbc_and_grep_and_fit, i))
    for future in concurrent.futures.as_completed(futures):
        pass   

### Cat rms trajectories
- The following is for heating and equilibration parts
- Use cat.traj to combine production run trajectories

In [ ]:
!gmx trjcat -f md*_rms.xtc -o {jobname}_5us500kf.xtc -settime 0
!gmx trjconv -f {jobname}_5us500kf.xtc -o {jobname}_5us5kf.xtc -skip 100 -ndec 5

In [ ]:
!gmx trjcat -f md*_noh.xtc -o {jobname}_5us500kf_noh.xtc -settime 0
!gmx trjconv -f {jobname}_5us500kf_noh.xtc -o {jobname}_5us5kf_noh.xtc -skip 100 -ndec 5

### Move data to next folder

In [ ]:
!cp {jobname}_5us500kf.xtc ../3.trajectory_data/
!cp {jobname}_5us5kf.xtc   ../3.trajectory_data/

## Visualize
```bash
pymol md0_c.pdb   md0_c.xtc
pymol md0_rms.pdb md0_rms.xtc
```